In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer
from transcriptor import get_transcript
import torch
import textwrap
from tqdm import tqdm
import re

c:\Users\farns\anaconda3\envs\transformer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def summarize(transcript, percent_len):
    
    model_name = 'sshleifer/distilbart-xsum-12-1'
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
    
    # transcript = get_transcript(url) 
    
    target_char_len = int(len(transcript) * percent_len)
    curr_char_len = target_char_len + 1
    
    print(f'Transcript Length:\t{len(transcript)} Target Char Length:\t{target_char_len}')
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=700,
        chunk_overlap=140
    )

    chunks = text_splitter.split_text(transcript)
    print(f'Num Initial Chunks:\t{len(chunks)}')
    
    while curr_char_len > target_char_len:
        count = 0
        chunk_summaries = []
        for chunk in tqdm(chunks):
            inputs = tokenizer(chunk, return_tensors='pt', max_length=1024, truncation=True).to(device)
            outputs = model.generate(inputs['input_ids'], max_length=200, min_length=40, length_penalty=2, num_beams=4)
            chunk_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

            chunk_summaries.append(chunk_summary)
            
        summary = ''.join([summary for summary in chunk_summaries])
        summary = re.sub(r'\s+\.', '.', text)
        curr_char_len = len(summary)
        print(f'Curr Char Length:\t{curr_char_len}')
        
        if curr_char_len < target_char_len:
            return summary
        
        chunks = text_splitter.split_text(summary)
        count += 1
        print(f'Num chunks after {count} iterations:\t{chunks}')

In [ ]:
t = get_transcript('https://www.youtube.com/watch?v=k0EdghetCMc')
print('Transcription Completed!')